In [ ]:
using Images
# using GLMakie   # not needed, VideoIO.playvideo not working
using VideoIO
using OpenCV; const cv = OpenCV

In [ ]:
img_CHW(img) = Images.channelview(img)
img_HWC(img) = Images.permutedims(img_CHW(img), (2, 3, 1))
img_CWH(img) = Images.permutedims(img_CHW(img), (1, 3, 2))


In [ ]:
function filesWithinFolder(folder, prefix, ext)
    cutoffName = length(prefix) + 1
    files = readdir(folder)
    extFiles = [file for file in files if occursin( ext, file )]
    sortedExtFiles = sort( [ parse(Int, file[cutoffName : end-4])  for file in extFiles ] )
    sortedExtFiles = string.(folder, "/", prefix, sortedExtFiles, ext)
end

In [ ]:
# get files for processing
folder  = "../../bags/video"
pngFiles = filesWithinFolder(folder, "img",".png");

In [ ]:
# load image sequence on an array stack
imgStack = [ Images.load(imgname) for imgname in pngFiles ];
typeof(imgStack[1])

In [ ]:
# https://trac.ffmpeg.org/wiki/Encode/H.264
# https://docs.juliahub.com/VideoIO/tZxJ7/1.0.0/writing/#Encoder-Options
encoderOptions = (crf=23,
                  preset="medium")

In [ ]:
# save image stack as video
VideoIO.save("video.mp4", imgStack;
             #codec_name="libx264",
             framerate=25,
             encoder_options=encoderOptions)

In [ ]:
# load video
v = VideoIO.load("video.mp4")
numberOfFrames = length(v)

In [ ]:
# playing with VideoIO not working, question placed at forums
# VideoIO.playvideo(v)

In [ ]:
fps = 30
frame_ms = round(1 / fps * 1000) |> Int32

In [ ]:
# play video with opencv
for i in 1:numberOfFrames
    cv.imshow("vaquinha", Float32.(img_CWH( v[i] )))
    cv.waitKey(frame_ms)
end
cv.destroyWindow("vaquinha")